In [52]:
from random import shuffle
from time import sleep

suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 'Nine':9, 'Ten':10, 'Jack':10,
         'Queen':10, 'King':10, 'Ace':11}

In [53]:
class Card():
    
    def __init__(self,suit,rank):
        self.suit = suit
        self.rank = rank
        self.value = values[rank]
        
    def __str__(self):
        return f"{self.rank} of {self.suit}"

In [54]:
class Deck():
    
    def __init__(self):
        self.cards = []
        for suit in suits:
            for rank in ranks:
                self.cards.append(Card(suit,rank))
                
    def __str__(self):
        if len(self.cards) != 1:
            return f"This deck have {len(self.cards)} cards"
        else:
            return f"This deck have {len(self.cards)} card"
        
    def shuffle(self):
        shuffle(self.cards)
        
    def deal_one(self):
        return self.cards.pop(0)

In [55]:
class Player():
    
    def __init__(self,name,money):
        self.name = name
        self.money = money
        self.cards = []
        self.aces = 0
        self.value = 0
        
    def __str__(self):
        return f"{self.name} has {self.money} dolars"
    
    def deposit(self,value):
        self.money += value
    
    def withdraw(self,value):
        if self.money >= value:
            self.money -= value
        else:
            value = self.money
            self.money = 0
        return value
        
    def hit(self,card):
        self.cards.append(card)
        if card.rank == "Ace":
            self.aces += 1
        self.value += card.value
        if self.value > 21 and self.aces != 0:
            self.value -= 10
            self.aces -= 1
            
    def reset_hand(self):
        self.cards = []
        self.aces = 0
        self.value = 0

In [56]:
class Dealer():
    
    def __init__(self):
        self.cards = []
        self.aces = 0
        self.value = 0
    
    def hit(self,card):
        self.cards.append(card)
        if card.rank == "Ace":
            self.aces += 1
        self.value += card.value
        if self.value > 21 and self.aces != 0:
            self.value -= 10
            self.aces -= 1
            
    def get_value(self):
        return self.value
    
    def get_first_card(self):
        if len(self.cards) > 0:
            return self.cards[0]
        else:
            return "Empty hand"
        
    def reset_hand(self):
        self.cards = []
        self.aces = 0
        self.value = 0

In [57]:
def check_input(string):
    inp = input(string)
    while inp != "y" and inp != "n":
        inp = input("Wrong input, the options are (y) and (n): ")
    return inp

In [62]:
dealer = Dealer()

print("Welcome to the game of BlackJack!\n")
sleep(0.5)

name = input("Please insert your name: ")
money = int(input("Now insert your total money: "))
player = Player(name,money)
sleep(0.5)

print(f"{player.name} starts the game with {player.money} dolars\n")
playing = "y"

while playing == "y":
    sleep(.5)
    intended_bet = int(input(f"{player.name}, how much do tou want to bet?: "))
    actual_bet = player.withdraw(intended_bet)
    if intended_bet == actual_bet:
        print(f"{player.name} bets {actual_bet} dolars, and is left with {player.money} dolars\n")
    else:
        print(f"{player.name} doesn't have this sum, and is betting all his money, {actual_bet} dolars\n")
    
    deck = Deck()
    deck.shuffle()
    
    sleep(0.5)
    print("Distributing initial hand\n")

    player.hit(deck.deal_one())
    dealer.hit(deck.deal_one())
    player.hit(deck.deal_one())
    dealer.hit(deck.deal_one())
    
    sleep(0.5)
    print(f"One of the dealer cards is {dealer.get_first_card()}\n")

    sleep(0.5)
    player_cards = ", ".join([card.__str__() for card in player.cards])
    print(f"{player.name} has [" + player_cards + f"] whith total value equal to {player.value}\n")
    bust = 0

    option = check_input(f"{player.name}, do you wish to hit(y) or stand(n)?: ")

    while option == "y" and bust == 0:
        sleep(0.5)
        player.hit(deck.deal_one())
        player_cards = ", ".join([card.__str__() for card in player.cards])
        if player.value > 21:
            print(f"{player.name} has [" + player_cards + f"] and busted!\n")
            print(f"Dealer wins!")
            print(f"With this loss, {player.name} has {player.money} dolars\n")
            bust = 1
        else:
            print(f"{player.name} has [" + player_cards + f"] whith total value equal to {player.value}\n")
            option = check_input(f"{player.name}, do you wish to hit(y) or stand(n)?: ")
    print("")

    if bust == 0:
        sleep(0.5)
        dealer_cards = ", ".join([card.__str__() for card in dealer.cards])
        print(f"Dealer has [" + dealer_cards + f"] whith total value equal to {dealer.value}\n")
        while dealer.value < 17:
            sleep(0.5)
            print("Dealer hits")
            dealer.hit(deck.deal_one())
            dealer_cards = ", ".join([card.__str__() for card in dealer.cards])
            print(f"Dealer has [" + dealer_cards + f"] whith total value equal to {dealer.value}\n")
        if dealer.value > 21:
            print(f"Dealer has busted!\n")
            print(f"{player.name} wins!")
            player.deposit(actual_bet*2)
            print(f"With this win, {player.name} has {player.money} dolars\n")
        else:
            sleep(0.5)
            print("Dealer stands\n")
            print(f"{player.name} has a total value of {player.value} against a total value of {dealer.value} from the dealer\n")
            if player.value > dealer.value:
                print(f"{player.name} wins!\n")
                player.deposit(actual_bet*2)
                print(f"With this win, {player.name} has {player.money} dolars\n")
            elif player.value < dealer.value:
                print(f"Dealer wins!\n")
                print(f"With this loss, {player.name} has {player.money} dolars\n")
            else:
                print(f"The round is tied\n")
                player.deposit(actual_bet)
                print(f"With this tie, {player.name} has {player.money} dolars\n")
    
    sleep(0.5)
    if player.money > 0:
        playing = check_input(f"{player.name}, do you wish to play again? (y) or (n): ")
        player.reset_hand()
        dealer.reset_hand()
    else:
        print(f"{player.name} doesn't have money to play again\n")
        playing = "n"
    
sleep(0.5)    
print(f"End of the game! {player.name} leaves with {player.money} dolars")

Welcome to the game of BlackJack!

Please insert your name: Daniel
Now insert your total money: 5000
Daniel starts the game with 5000 dolars

Daniel, how much do tou want to bet?: 2000
Daniel bets 2000 dolars, and is left with 3000 dolars

Distributing initial hand

One of the dealer cards is Eight of Spades

Daniel has [Nine of Hearts, Two of Spades] whith total value equal to 11

Daniel, do you wish to hit(y) or stand(n)?: y
Daniel has [Nine of Hearts, Two of Spades, Two of Clubs] whith total value equal to 13

Daniel, do you wish to hit(y) or stand(n)?: y
Daniel has [Nine of Hearts, Two of Spades, Two of Clubs, Five of Spades] whith total value equal to 18

Daniel, do you wish to hit(y) or stand(n)?: n

Dealer has [Eight of Spades, Nine of Spades] whith total value equal to 17

Dealer stands

Daniel has a total value of 18 against a total value of 17 from the dealer

Daniel wins!

With this win, Daniel has 7000 dolars

Daniel, do you wish to play again? (y) or (n): y
Daniel, how muc